# Install

In [ ]:
!pip3 install pshmodule

In [ ]:
!pip3 install pickle5

In [ ]:
!pip3 install pandas==1.5.0

In [ ]:
!pip3 install swifter

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Load New

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/MemeProject/src/doc2vec/data_augmentation')
print(sys.path)

['/content', '/env/python', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.8/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/MemeProject/src/doc2vec/data_augmentation']


In [ ]:
import pandas as pd
import config as cfg
from tqdm import tqdm
import swifter
from pshmodule.utils import filemanager as fm

In [ ]:
df1 = fm.load(cfg.data_path_v1)
df2 = fm.load(cfg.data_path_v2)

extension : .xlsx
Loaded 21962 records from drive/MyDrive/MemeProject/data/row_data/meme_v1.xlsx
extension : .xlsx
Loaded 14640 records from drive/MyDrive/MemeProject/data/row_data/meme_v2.xlsx


In [ ]:
new_header = df1.iloc[1]
df1 = df1[2:]
df1.columns = new_header

Merge - 관리 번호 df1 A, E 제외, df2 A, E

In [ ]:
df1 = df1[['관리번호', 'utt',	'발화구분',	'발화문',	'ARG1',	'ARG_op']]
df2 = df2[['관리번호', 'utt',	'발화구분',	'발화문',	'ARG1',	'ARG_op']]

In [ ]:
print(len(df1))
print(len(df2))

21960
14640


In [ ]:
df_A = df2[df2['관리번호'].str.startswith('A')]
df_E = df2[df2['관리번호'].str.startswith('E')]

In [ ]:
df = pd.concat([df_A, df1, df_E])
df.reset_index(inplace=True, drop=True)

Numbering

In [ ]:
# no
num_list = [i+1 for i in range(len(df))]
df['num'] = num_list

# group no
g_list = []
num = 1
for i in range(len(df)):
  if i % 24 == 0 and i != 0:
    num += 1
  g_list.append(num)

df['g_num'] = g_list

In [ ]:
df = df[['num', 'g_num', 'utt', '발화구분', '발화문', 'ARG1', 'ARG_op']]
df.rename(columns={'발화구분':'type', '발화문':'u', 'ARG1':'arg1', 'ARG_op':'arg_op'}, inplace=True)
df = df.fillna('')

<ipython-input-15-79048a69353d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'발화구분':'type', '발화문':'u', 'ARG1':'arg1', 'ARG_op':'arg_op'}, inplace=True)


In [ ]:
df.tail()

,num,g_num,utt,type,u,arg1,arg_op
36595,36596,1525,20,CPO,,,
36596,36597,1525,21,CPO,,,
36597,36598,1525,22,CPO,,,
36598,36599,1525,23,CPO,,,
36599,36600,1525,24,CPO,,,


Reshape

In [ ]:
result = []
temp = []

for i in tqdm(df.iterrows()):
  temp.append([i[1]['num'], i[1]['g_num'], i[1]['utt'], i[1]['type'], i[1]['u'], i[1]['arg1'], i[1]['arg_op']])
  
  if i[1]['num'] % 24 == 0 and i[1]['num'] != 0:
    temp_u = temp[:6]
    temp_cpr = temp[6:12]
    temp_mm = temp[12:18]
    temp_cpo = temp[18:24]

    for u in temp_u:
      # 밈 조합
      for cpr in temp_cpr:
        for mm in temp_mm:
          for cpo in temp_cpo:
            meme = "{} {} {}".format(cpr[4], mm[4], cpo[4]).strip()
            result.append([u[0], u[1], u[2], u[4], meme, u[5], u[6]])
    # 24개 씩 초기화
    temp = []

36600it [00:07, 4662.84it/s]


In [ ]:
len(result)

1976400

In [ ]:
df = pd.DataFrame(result, columns=[['num', 'g_num', 'u_num', 'u', 'meme', 'arg1', 'arg_op']])

In [ ]:
df.head()

,num,g_num,u_num,u,meme,arg1,arg_op
0,1,1,1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 내 것도 곁들인...,중간고사,
1,1,1,1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 내 것도 곁들인...,중간고사,
2,1,1,1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 내 것도 곁들인...,중간고사,
3,1,1,1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 내 것도 곁들인...,중간고사,
4,1,1,1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 내 것도 곁들인...,중간고사,


arg_op 쉼표 기준으로 쪼개기

In [ ]:
df['arg_op1'] = df.arg_op.swifter.apply(lambda x: x.str.split(',').str[0])
df['arg_op2'] = df.arg_op.swifter.apply(lambda x: x.str.split(',').str[1])

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
df.fillna('', inplace=True)

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.reset_index(inplace=True, drop=True)

In [ ]:
df = df[['num', 'g_num', 'u_num', 'u', 'meme', 'arg1', 'arg_op1', 'arg_op2']]

In [ ]:
df.tail()

In [ ]:
df.iloc[430:440]

Temp Save

In [ ]:
fm.save(cfg.origin_ref, df)

Saved 181704 records


# Replace & Augmentation

In [1]:
import sys
sys.path.append('/content/drive/MyDrive/MemeProject/src/doc2vec/data_augmentation')
print(sys.path)

['/content', '/env/python', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.8/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/MemeProject/src/doc2vec/data_augmentation']


In [2]:
from tqdm import tqdm
import config as cfg
import pandas as pd
from pshmodule.utils import filemanager as fm

In [3]:
df = fm.load(cfg.origin_ref)

extension : .pickle
Loaded 181704 records from drive/MyDrive/MemeProject/data/augmentation/origin_ref.pickle


In [4]:
df.head()

,num,g_num,u_num,u,meme,arg1,arg_op1,arg_op2
0,1,1,1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 내 것도 곁들인...,중간고사,,
1,1,1,1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 내 몫을 제일 많이 곁들인...,중간고사,,
2,1,1,1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 대춘이 몫도 곁들인...,중간고사,,
3,1,1,1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 배보다 배꼽이 더 큰 걸 곁들인...,중간고사,,
4,1,1,1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 과소비를 곁들인...,중간고사,,


##### augmentation

In [6]:
num = []
g_num = []
u_num = []
u = []
meme = []
arg1 = []
arg_op1 = []
arg_op2 = []
result = []

for i in tqdm(df.iterrows()):
  if g_num and g_num[-1] != i[1]['g_num']:
    # 문장 6개씩 조합
    set_arg1 = set(arg1)
    set_argop1 = set(arg_op1)
    set_argop2 = set(arg_op2)
    set_arg1.discard('')
    set_argop1.discard('')
    set_argop2.discard('')

    for z in zip(num, g_num, u_num, u, meme):
      # arg1 교체
      result.append([z[0], z[1], z[2], z[3], z[4], '원본', '-', '-'])
      for s in set_arg1:
        if s in z[3]:
          temp_arg1 = [i for i in set_arg1 if i != s]
          for t in temp_arg1:
            z1 = z[3].replace(s, t)
            result.append([z[0], z[1], z[2], z1, z[4], t, '-', '-'])

            # arg_op1 교체
            for ss in set_argop1:
              if ss in z[3]:
                temp_argop1 = [i for i in set_argop1 if i != ss]
                for tt in temp_argop1:
                  z2 = z1.replace(ss, tt)
                  result.append([z[0], z[1], z[2], z2, z[4], t, tt, '-'])

                  # arg_op2 교체
                  for sss in set_argop2:
                    if sss in z[3]:
                      temp_argop2 = [i for i in set_argop1 if i != sss]
                      for ttt in temp_argop2:
                        z3 = z2.replace(sss, ttt)
                        result.append([z[0], z[1], z[2], z3, z[4], t, tt, ttt])

    # init
    num.clear()
    g_num.clear()
    u_num.clear()
    u.clear()
    meme.clear()
    arg1.clear()
    arg_op1.clear()
    arg_op2.clear()
  
  # data insert
  num.append(i[1]['num'])
  g_num.append(i[1]['g_num'])
  u_num.append(i[1]['u_num'])
  u.append(i[1]['u'])
  meme.append(i[1]['meme'])
  arg1.append(i[1]['arg1'])
  arg_op1.append(i[1]['arg_op1'])
  arg_op2.append(i[1]['arg_op2'])

181704it [00:46, 3939.58it/s]


In [7]:
print(df.shape)
print(len(result))

(181704, 8)
1177990


save reference data set

In [8]:
df_result = pd.DataFrame(result, columns=[['num', 'g_num', 'u_num', 'utterance', 'meme', 'arg1', 'arg_op1', 'arg_op2']])

In [9]:
df_result.drop_duplicates(inplace=True)
df_result.reset_index(inplace=True, drop=True)
df_result.fillna('', inplace=True)

In [10]:
df_result.tail(50)

,num,g_num,u_num,utterance,meme,arg1,arg_op1,arg_op2
1167932,36558,1524,6,울 엄마한테 눈썹반영구문신 시술받을 거다,탈룰라 조심해야겠네 희희,눈썹반영구문신,울 엄마한테,-
1167933,36558,1524,6,울엄마한테 눈썹반영구문신 시술받을 거다,탈룰라 조심해야겠네 희희,눈썹반영구문신,울엄마한테,-
1167934,36558,1524,6,엄마한테 눈썹문신 시술받을 거다,오~ 탈룰라 조심하겠음 희희,원본,-,-
1167935,36558,1524,6,엄마한테 눈썹타투 시술받을 거다,오~ 탈룰라 조심하겠음 희희,눈썹타투,-,-
1167936,36558,1524,6,울 엄마한테 눈썹타투 시술받을 거다,오~ 탈룰라 조심하겠음 희희,눈썹타투,울 엄마한테,-
1167937,36558,1524,6,울엄마한테 눈썹타투 시술받을 거다,오~ 탈룰라 조심하겠음 희희,눈썹타투,울엄마한테,-
1167938,36558,1524,6,엄마한테 눈썹반영구 시술받을 거다,오~ 탈룰라 조심하겠음 희희,눈썹반영구,-,-
1167939,36558,1524,6,울 엄마한테 눈썹반영구 시술받을 거다,오~ 탈룰라 조심하겠음 희희,눈썹반영구,울 엄마한테,-
1167940,36558,1524,6,울엄마한테 눈썹반영구 시술받을 거다,오~ 탈룰라 조심하겠음 희희,눈썹반영구,울엄마한테,-
1167941,36558,1524,6,엄마한테 아이브로우문신 시술받을 거다,오~ 탈룰라 조심하겠음 희희,아이브로우문신,-,-


In [11]:
print(len(df_result))

1167982


In [12]:
fm.save(cfg.aug_ref, df_result)

Saved 1167982 records


##### 참고 - 이전 알고리즘

In [ ]:
# num = []
# g_num = []
# u_num = []
# u = []
# meme = []
# arg1 = []
# arg_op1 = []
# arg_op2 = []
# result = []

# for i in tqdm(df.iterrows()):
#   if g_num and g_num[-1] != i[1]['g_num']:
#     # 문장 6개씩 조합
#     set_arg1 = set(arg1)
#     set_argop1 = set(arg_op1)
#     set_argop2 = set(arg_op2)
#     set_arg1.discard('')
#     set_argop1.discard('')
#     set_argop2.discard('')

#     for z in zip(num, g_num, u_num, u, meme):
#       # arg1 교체
#       arg_split = z[3].split()
#       for k, s in enumerate(arg_split):
#         result.append([z[0], z[1], z[2], z[3], z[4], '-', '-', '-'])
        
#         if s in set_arg1:
#           for w in set_arg1:
#             arg_split[k] = w

#             # arg_op1 교체
#             for kk, ss in enumerate(arg_split):
#               result.append([z[0], z[1], z[2], ' '.join(arg_split), z[4], w, '-', '-'])
#               if ss in set_argop1:
#                 for ww in set_argop1:
#                   arg_split[kk] = ww
                  
#                   # arg_op2 교체
#                   for kkk, sss in enumerate(arg_split):
#                     result.append([z[0], z[1], z[2], ' '.join(arg_split), z[4], w, ww, '-'])
#                     if sss in set_argop2:
#                       for www in set_argop2:
#                         arg_split[kkk] = www
#                         result.append([z[0], z[1], z[2], ' '.join(arg_split), z[4], w, ww, www])

#     # init
#     num.clear()
#     g_num.clear()
#     u_num.clear()
#     u.clear()
#     meme.clear()
#     arg1.clear()
#     arg_op1.clear()
#     arg_op2.clear()
  
#   # data insert
#   num.append(i[1]['num'])
#   g_num.append(i[1]['g_num'])
#   u_num.append(i[1]['u_num'])
#   u.append(i[1]['u'])
#   meme.append(i[1]['meme'])
#   arg1.append(i[1]['arg1'])
#   arg_op1.append(i[1]['arg_op1'])
#   arg_op2.append(i[1]['arg_op2'])

181704it [00:53, 3421.30it/s]


# Report Save

In [13]:
import sys
sys.path.append('/content/drive/MyDrive/MemeProject/src/doc2vec/data_augmentation')
print(sys.path)

['/content', '/env/python', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.8/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/MemeProject/src/doc2vec/data_augmentation', '/content/drive/MyDrive/MemeProject/src/doc2vec/data_augmentation']


In [14]:
from tqdm import tqdm
import config as cfg
import pandas as pd
from pshmodule.utils import filemanager as fm

In [15]:
df = fm.load(cfg.origin_ref)
df_result = fm.load(cfg.aug_ref)

extension : .pickle
Loaded 181704 records from drive/MyDrive/MemeProject/data/augmentation/origin_ref.pickle
extension : .pickle
Loaded 1167982 records from drive/MyDrive/MemeProject/data/augmentation/aug_ref.pickle


In [16]:
df_report = df_result[['num', 'g_num', 'u_num', 'utterance',	'arg1',	'arg_op1', 'arg_op2']]

In [17]:
df_report.drop_duplicates(inplace=True)
df_report.reset_index(inplace=True, drop=True)

<ipython-input-17-c655761fba79>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_report.drop_duplicates(inplace=True)


In [18]:
df_report.rename(columns={'num':'일련번호', 'g_num':'그룹 번호', 'u_num':'utt 번호', 'utterance':'발화문'}, inplace=True)

<ipython-input-18-4ba571b1eaea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_report.rename(columns={'num':'일련번호', 'g_num':'그룹 번호', 'u_num':'utt 번호', 'utterance':'발화문'}, inplace=True)


In [19]:
df_report.shape

(59364, 7)

In [20]:
fm.save(cfg.aug_report, df_report)

Saved 59364 records
